In [361]:
import cv2 as cv
from enum import Enum
import numpy as np
import builtins

computeErrorSurface:
    img1 is always on top or on left of img2.

In [362]:
class Direction(Enum):
    VERTICAL = 0
    HORIZONTAL = 1

In [363]:
def computeErrorSurface(img1, img2, boundarySize: int, direction: Direction):
    border1 = border2 = None
    if direction == Direction.HORIZONTAL:
        border1 = img1[-boundarySize-1:-1,:]
        border2 = img2[0:boundarySize,:]
    else:
        border1 = img1[:,-boundarySize-1:-1]
        border2 = img2[:,0:boundarySize]

    diff = np.subtract(border1, border2)

    squareDiff = np.square(diff)

    ret = np.zeros((squareDiff.shape[0], squareDiff.shape[1]),dtype=np.uint32)

    for i in range(ret.shape[0]):
        for j in range(ret.shape[1]):
            ret[i,j] = np.sum(squareDiff[i,j])

    ret = np.sqrt(ret)
    return cv.normalize(ret,None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_8U)


In [364]:
def verticalNeighbors(currentPos, errorBoundary):

    neighbors = []

    if(currentPos[0] != errorBoundary.shape[0]):
        neighbors.append((currentPos[0] - 1, currentPos[1]))
        if(currentPos[1] != 0):
            neighbors.append((currentPos[0] - 1, currentPos[1] - 1))
        if(currentPos[1] != errorBoundary.shape[1] - 1):
            neighbors.append((currentPos[0] - 1, currentPos[1] + 1))
    else:
        for i in range(errorBoundary.shape[1]):
            neighbors.append((currentPos[0] - 1, i))

    return neighbors

def horizontalNeighbors(currentPos, errorBoundary):

    neighbors = []

    if(currentPos[1] != errorBoundary.shape[1]):
        neighbors.append((currentPos[0], currentPos[1] - 1))
        if(currentPos[0] != 0):
            neighbors.append((currentPos[0] - 1, currentPos[1] - 1))
        if(currentPos[0] != errorBoundary.shape[0] - 1):
            neighbors.append((currentPos[0] + 1, currentPos[1] - 1))
    else:
        for i in range(errorBoundary.shape[0]):
            neighbors.append((i, currentPos[1] - 1))

    return neighbors

In [365]:
# function heavily inspired by Ben Alex Keen's implementation: https://benalexkeen.com/implementing-djikstras-shortest-path-algorithm-with-python/
def minimumBoundaryCutDjikstras(errorBoundary, direction: Direction):
    initial = [0,0]
    initial[direction.value] = errorBoundary.shape[direction.value]
    initial = tuple(initial)
    
    neighborFunction = verticalNeighbors

    if(direction == Direction.HORIZONTAL):
        neighborFunction = horizontalNeighbors


    print(neighborFunction.__name__)

    shortestPaths = {initial: (None, 0)}
    currentNode = initial
    visited = set()
    
    while currentNode[direction.value] != 0:
        visited.add(currentNode)
        destinations = neighborFunction(currentNode, errorBoundary)
        weightToCurrentNode = shortestPaths[currentNode][1]

        for nextNode in destinations:
            weight = errorBoundary[nextNode[0], nextNode[1]] + weightToCurrentNode
            if nextNode not in shortestPaths:
                shortestPaths[nextNode] = (currentNode, weight)
            else:
                currentShortestWeight = shortestPaths[nextNode][1]
                if currentShortestWeight > weight:
                    shortestPaths[nextNode] = (currentNode, weight)
        
        nextDestinations = {node: shortestPaths[node] for node in shortestPaths if node not in visited}
        currentNode = builtins.min(nextDestinations, key=lambda k: nextDestinations[k][1])
    
    path = []
    while currentNode is not None:
        path.append(currentNode)
        nextNode = shortestPaths[currentNode][0]
        currentNode = nextNode

    path = path[-2::-1]
    return path

In [366]:
def show(names, imgs, scaling = 1):
    for i in range(len(names)):
        cv.imshow(names[i], cv.resize(imgs[i], (imgs[i].shape[1] * scaling, imgs[i].shape[0] * scaling)))
    cv.waitKey(0)
    cv.destroyAllWindows()

In [367]:
def computeMinimumBoundaryCut(img1, img2, boundarySize, direction):
    e = computeErrorSurface(img1, img2, boundarySize, direction)
    minBCut = minimumBoundaryCutDjikstras(e, direction)
    return (e,minBCut)

In [368]:
def applyWithMask(fullImg, patch, position, boundarySize, cutUp = None, cutLeft = None):
    size = patch.shape[0] - boundarySize
    pixelPosition = [position[0]*size, position[1]*size]
    patchForPasting = patch.copy()

    if(cutUp != None):
        for point in cutUp:
            patchForPasting[point[0],:point[1]] = 0
            fullImg[pixelPosition[0] + point[0], pixelPosition[1] + point[1]:] = 0
        

    if(cutLeft != None):
        for point in cutLeft:
            patchForPasting[:point[0],point[1]] = 0
            fullImg[pixelPosition[0] + point[0]:, pixelPosition[1] + point[1]] = 0

    fullImg[pixelPosition[0]:pixelPosition[0] + patch.shape[0], pixelPosition[1]:pixelPosition[1] + patch.shape[1]] += patchForPasting

In [371]:
img1 = cv.imread("img1.jpg", cv.IMREAD_COLOR)
img2 = cv.imread("img2.png", cv.IMREAD_COLOR)
borderSize = 50

e, minBCut = computeMinimumBoundaryCut(img1, img2, borderSize, Direction.VERTICAL)

minBCut3 = cv.merge((e,e,e))
#for point in minBCut:
minBCut3[point] = [0,255,0]

cellSize = img1.shape[0] - borderSize
canvas = np.zeros((cellSize*2 + borderSize, cellSize*2 + borderSize, 3), dtype=np.uint8)
applyWithMask(canvas, img1, (0,0), borderSize)
applyWithMask(canvas, img2, (0,1), borderSize, cutUp=minBCut)

show(["image 1", "image 2", "boundary error", "boundary cut", "canvas"], [img1, img2, e, minBCut3, canvas], 2)

verticalNeighbors
